In [25]:
# Imports
from tkinter import filedialog, Tk
from ultralytics import YOLO
import yaml
from os import path, listdir
from shutil import copytree, rmtree, copy
from vsrc.constants import DATA
%load_ext autoreload
%gui tk

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Helper Functions
def save_no_overwrite(file):
    suffix = 0
    name, extension = path.splitext(file)
    while path.exists(file):
        file = f"{name}_{suffix}{extension}"
        suffix += 1
    return file

def open_file():
    root = Tk()
    filename = filedialog.askopenfilename(title='Select Model Or Checkpoint', initialdir='./models/', filetypes=[('Model Files', '*.pt')])
    root.destroy()
    return filename

In [3]:
# Constants
CHECKPOINT_PATH, RECIPE_PATH = './models/checkpoints/', './models/recipes/'
IMAGE_SIZE = 250

# Derived Constants

### Identifier

In [28]:
# Identifier Recipe
DATASET_FOLDER = './dataset/all'
YAML_RECIPE = {
    'path' : DATASET_FOLDER,
    'train' : 'train',
    'val' : 'test',
    'names' : [{ DATA[name]["num_label"] : name} for name in DATA],
}
# Copy all component data to all folder
if path.exists(DATASET_FOLDER): rmtree(DATASET_FOLDER)
# Get all component images from all folders
images, labels = [], []
for folder in listdir('./dataset'):
    if folder in ['template', 'test.png']: continue
    # Append all files in folder to images
    imgfiles = listdir(path.join('./dataset', folder, 'imgs'))
    labfiles = listdir(path.join('./dataset', folder, 'labels'))
    for file in imgfiles:
        images.append(path.join('./dataset', folder, 'imgs', file))
    for label in labfiles:
        labels.append(path.join('./dataset', folder, 'labels', label))
    print(f"Found {len(imgfiles)} images and {len(labfiles)} labels in {folder}")
# Copy all fules to new folder
copytree('./dataset/template', DATASET_FOLDER)
# Copy all images to new folder
for image in images:
    copy(image, path.join(DATASET_FOLDER, 'imgs'))
# Copy all labels to new folder
for label in labels:
    copy(label, path.join(DATASET_FOLDER, 'labels'))

Found 2 images and 1 labels in resistor


### Load Models

In [14]:
# Base Model
MODEL_PATH, MODEL_NAME = path.split(open_file())
# YAML Saving
YAML_FILEPATH = save_no_overwrite(path.join(RECIPE_PATH, f'{MODEL_NAME}.yaml'))
with open(YAML_FILEPATH, 'w') as f:
    yaml.dump(YAML_RECIPE, f)